# Predict tags on StackOverflow with linear models

In this assignment you will learn how to predict tags for posts from [StackOverflow](https://stackoverflow.com). To solve this task you will use multilabel classification approach.

### Libraries

In this task you will need the following libraries:
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [Pandas](https://pandas.pydata.org) — a library providing high-performance, easy-to-use data structures and data analysis tools for the Python
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [NLTK](http://www.nltk.org) — a platform to work with natural language.

### Data

You can find all data required for this assignment into the folder `/data`.

### Text preprocessing

For this assignment you will need to use a list of stop words. It can be downloaded from *nltk*:

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In this task you will deal with a dataset of post titles from StackOverflow. You are provided a split to 3 sets: *train*, *validation* and *test*. All corpora (except for *test*) contain titles of the posts and corresponding tags (100 tags are available). The *test* set doesn't contain answers. Upload the corpora using *pandas* and look at the data:

In [0]:
from ast import literal_eval
import pandas as pd
import numpy as np

Literal_eval package takes care of the preprocessing for the string so that it can be used in python. To know more on literal_eval please see the below documentation <br>
https://kite.com/python/docs/ast.literal_eval

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Stats_NLP/')

## Task 1: Create training, testing and validation data from the files given. Use title to be the independent variable and tags to be the dependent variable ( 5 points)

Note: Ensure you apply literal_eval function on the tags column to ensure all the tags are readable in python

In [5]:
trainDf = pd.read_csv("train.tsv",delimiter='\t',encoding='utf-8')
trainDf.head()

,title,tags
0,How to draw a stacked dotplot in R?,['r']
1,mysql select all records where a datetime fiel...,"['php', 'mysql']"
2,How to terminate windows phone 8.1 app,['c#']
3,get current time in a specific country via jquery,"['javascript', 'jquery']"
4,Configuring Tomcat to Use SSL,['java']


In [6]:
testDf = pd.read_csv("test.tsv", delimiter="\t", encoding='utf-8')
testDf.head()

,title
0,Warning: mysql_query() expects parameter 2 to ...
1,get click coordinates from <input type='image'...
2,How to implement cloud storage for media asset...
3,What is catcomplete in jQuery's autocomplete p...
4,Error building Android app with Cordova 3.1 CLI


In [7]:
validationDf = pd.read_csv("validation.tsv", delimiter="\t", encoding='utf-8')
validationDf.head()

,title,tags
0,Why odbc_exec always fail?,"['php', 'sql']"
1,Access a base classes variable from within a c...,['javascript']
2,"Content-Type ""application/json"" not required i...","['ruby-on-rails', 'ruby']"
3,Sessions in Sinatra: Used to Pass Variable,"['ruby', 'session']"
4,"Getting error - type ""json"" does not exist - i...","['ruby-on-rails', 'ruby', 'json']"


In [0]:
import ast
trainTags = set()
for index, row in trainDf.iterrows():
  x = ast.literal_eval(row['tags'])
  trainTags.update(x)

In [9]:
len(trainTags)

100

In [10]:
print(trainTags)

{'json', 'ruby', 'csv', 'datetime', 'jquery', 'winforms', 'generics', 'ruby-on-rails', 'c#', 'ajax', 'visual-studio', 'django', 'excel', 'arrays', 'c', 'sql', 'performance', 'linux', 'xml', 'list', 'node.js', 'jsp', 'spring-mvc', 'visual-studio-2010', 'database', 'regex', 'python', 'oop', 'ruby-on-rails-3', 'eclipse', 'css', 'session', 'parsing', 'android', 'numpy', 'asp.net-mvc', 'codeigniter', 'rest', 'osx', 'sorting', 'google-maps', 'mysql', 'spring', 'facebook', 'objective-c', 'opencv', 'javascript', 'html', 'vb.net', 'forms', 'java', 'date', 'loops', 'php', 'hibernate', 'image', 'ios', 'xaml', 'apache', 'laravel', 'string', 'asp.net', 'uitableview', 'class', 'python-2.7', 'wcf', 'linq', 'python-3.x', '.net', 'html5', 'pointers', 'xcode', 'function', 'web-services', 'swing', 'pandas', 'unit-testing', 'maven', 'twitter-bootstrap', 'angularjs', 'dom', 'wpf', 'mongodb', 'iphone', 'entity-framework', 'file', 'servlets', 'wordpress', 'algorithm', 'selenium', 'swift', 'sockets', 'multith

In [0]:
validationTags = set()
for index, row in validationDf.iterrows():
  x = ast.literal_eval(row['tags'])
  validationTags.update(x)

In [12]:
len(validationTags)

100

In [13]:
print(validationTags)

{'json', 'ruby', 'datetime', 'csv', 'jquery', 'winforms', 'generics', 'ruby-on-rails', 'c#', 'ajax', 'visual-studio', 'django', 'excel', 'arrays', 'sql', 'c', 'performance', 'linux', 'xml', 'node.js', 'list', 'jsp', 'spring-mvc', 'visual-studio-2010', 'database', 'regex', 'python', 'oop', 'ruby-on-rails-3', 'eclipse', 'css', 'session', 'parsing', 'android', 'numpy', 'asp.net-mvc', 'codeigniter', 'rest', 'osx', 'sorting', 'google-maps', 'mysql', 'spring', 'facebook', 'objective-c', 'opencv', 'javascript', 'html', 'vb.net', 'forms', 'java', 'date', 'loops', 'php', 'hibernate', 'image', 'ios', 'xaml', 'apache', 'laravel', 'string', 'asp.net', 'uitableview', 'class', 'python-2.7', 'wcf', 'linq', 'python-3.x', '.net', 'pointers', 'html5', 'xcode', 'function', 'web-services', 'swing', 'pandas', 'unit-testing', 'maven', 'twitter-bootstrap', 'angularjs', 'dom', 'wpf', 'mongodb', 'iphone', 'entity-framework', 'file', 'servlets', 'wordpress', 'algorithm', 'selenium', 'swift', 'sockets', 'multith

## Task 2 (Pre-processing). Implement the function *text_prepare* following the instructions. After that, run the function *test_test_prepare* to test it on tiny cases. (10 points)

One of the most known difficulties when working with natural data is that it's unstructured. For example, if you use it "as is" and extract tokens just by splitting the titles by whitespaces, you will see that there are many "weird" tokens like *3.5*, *?*,  *{}*, etc. To prevent the problems, it's usually useful to prepare the data in a custom way


In [0]:
import re

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it.
    text = text.lower() ### Write a code which can change the input text to lowercase.
    text = REPLACE_BY_SPACE_RE.sub(" ",text) ### Write a code which replaces REPLACE_BY_SPACE_RE (above mentioned) symbols by space in text
    text = BAD_SYMBOLS_RE.sub("",text) ### Write a code which deletes symbols which are in BAD_SYMBOLS_RE (above mentioned) from text
    
    ### Write a code which deletes stopwords from text
    querywords = text.split()
    resultwords  = [word for word in querywords if word.lower() not in STOPWORDS]
    text = ' '.join(resultwords)
    return text

In [0]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function

In [18]:
test_text_prepare()

'Basic tests are passed.'

*Note: You should pass the above test to ensure the text preprocessing is done before our analysis*

Now we can preprocess the titles using function *text_prepare* and  making sure that the headers don't have bad symbols:

In [0]:
trainCleanDf = trainDf.copy(deep=True)
testCleanDf = testDf.copy(deep=True)
validationCleanDf = validationDf.copy(deep=True)

In [0]:
for index, row in trainCleanDf.iterrows():
  row['title'] = text_prepare(row['title'])

In [0]:
for index, row in testCleanDf.iterrows():
  row['title'] = text_prepare(row['title'])

In [0]:
for index, row in validationCleanDf.iterrows():
  row['title'] = text_prepare(row['title'])

Print the top 5 elements in x_train

In [23]:
trainCleanDf.head()

,title,tags
0,draw stacked dotplot r,['r']
1,mysql select records datetime field less speci...,"['php', 'mysql']"
2,terminate windows phone 81 app,['c#']
3,get current time specific country via jquery,"['javascript', 'jquery']"
4,configuring tomcat use ssl,['java']


## Task 2 (WordsTagsCount) - Find 3 most popular tags and 3 most popular words in the train data. - 5 points

Note: The words which appear the most are considered as popular in this case!

In [24]:
import ast
tagList = []
for index, row in trainDf.iterrows():
  x = ast.literal_eval(row['tags'])
  tagList.extend(x)
  
len(tagList)

194219

In [25]:
print(tagList[:15])

['r', 'php', 'mysql', 'c#', 'javascript', 'jquery', 'java', 'ruby-on-rails', 'ruby', 'ruby-on-rails-3', 'json', 'ruby', 'java', 'spring', 'spring-mvc']


In [26]:
df = pd.DataFrame(tagList, columns={"tag"})
df.head()

,tag
0,r
1,php
2,mysql
3,c#
4,javascript


In [27]:
df.shape

(194219, 1)

In [28]:
# top 3 tags
grouped = df.groupby('tag')['tag'].count()
grouped.sort_values(ascending=False)[:3]

tag
javascript    19078
c#            19077
java          18661
Name: tag, dtype: int64

We are assuming that *tags_counts* and *words_counts* are dictionaries like `{'some_word_or_tag': frequency}`. After applying the sorting procedure, results will be look like this: `[('most_popular_word_or_tag', frequency), ('less_popular_word_or_tag', frequency), ...]`.

eg: 
Tag 1 - 100 Tag 2 - 65 Tag 3 - 250 <br>
after sorting looks like, <br>
Tag 3 - 250 Tag 1 - 100 Tag 2 - 65

In [29]:
tagDict = grouped.sort_values(ascending=False).to_dict()
for x in list(tagDict)[0:3]:
    print ("key: {}, frequency: {} ".format(x,  tagDict[x]))

key: javascript, frequency: 19078 
key: c#, frequency: 19077 
key: java, frequency: 18661 


In [30]:
wordList = []
for index, row in trainCleanDf.iterrows():
  x = row['title'].split()
  wordList.extend(x)
  
len(wordList)

565425

In [31]:
wdf = pd.DataFrame(wordList, columns={"word"})
wdf.head()

,word
0,draw
1,stacked
2,dotplot
3,r
4,mysql


In [33]:
# popular words
grouped = wdf.groupby('word')['word'].count()
grouped.sort_values(ascending=False)[:3]

word
using    8278
php      5614
java     5501
Name: word, dtype: int64

In [34]:
wordDict = grouped.sort_values(ascending=False).to_dict()
for x in list(wordDict)[0:3]:
    print ("key: {}, frequency: {} ".format(x,  wordDict[x]))

key: using, frequency: 8278 
key: php, frequency: 5614 
key: java, frequency: 5501 


## Task - 3 Transforming text to a vector (10 points)

Machine Learning algorithms work with numeric data and we cannot use the provided text data "as is". There are many ways to transform text data to numeric vectors. In this task you will try to use two of them.

#### Bag of words

One of the well-known approaches is a *bag-of-words* representation. To create this transformation, follow the steps:
1. Find *N* most popular words in train corpus and numerate them. Now we have a dictionary of the most popular words.
2. For each title in the corpora create a zero vector with the dimension equals to *N*.
3. For each text in the corpora iterate over words which are in the dictionary and increase by 1 the corresponding coordinate.

Let's try to do it for a toy example. Imagine that we have *N* = 4 and the list of the most popular words is 

    ['hi', 'you', 'me', 'are']

Then we need to numerate them, for example, like this: 

    {'hi': 0, 'you': 1, 'me': 2, 'are': 3}

And we have the text, which we want to transform to the vector:

    'hi how are you'

For this text we create a corresponding zero vector 

    [0, 0, 0, 0]
    
And iterate over all words, and if the word is in the dictionary, we increase the value of the corresponding position in the vector:

    'hi':  [1, 0, 0, 0]
    'how': [1, 0, 0, 0] # word 'how' is not in our dictionary
    'are': [1, 0, 0, 1]
    'you': [1, 1, 0, 1]

The resulting vector will be 

    [1, 1, 0, 1]
   
Implement the described encoding in the function *my_bag_of_words* with the size of the dictionary equals to 5000. To find the most common words use train data. You can test your code using the function *test_my_bag_of_words*.

In [0]:
wordsToIndex = { list(wordDict.keys())[i]:i for i in range(0, 5000) }
indexOfWords = { j:list(wordDict.keys())[j] for j in range(0, 5000) }

In [36]:
print(wordsToIndex)

{'using': 0, 'php': 1, 'java': 2, 'file': 3, 'javascript': 4, 'error': 5, 'get': 6, 'c#': 7, 'python': 8, 'string': 9, 'array': 10, 'data': 11, 'jquery': 12, 'value': 13, 'function': 14, 'object': 15, 'class': 16, 'use': 17, 'c++': 18, 'list': 19, 'method': 20, 'multiple': 21, 'text': 22, 'image': 23, 'page': 24, 'rails': 25, 'form': 26, 'create': 27, 'html': 28, 'type': 29, 'code': 30, 'set': 31, 'working': 32, 'database': 33, 'add': 34, 'convert': 35, 'mysql': 36, 'json': 37, 'c': 38, 'server': 39, 'variable': 40, 'android': 41, 'values': 42, 'aspnet': 43, 'way': 44, 'application': 45, 'change': 46, 'button': 47, 'find': 48, 'django': 49, 'one': 50, 'without': 51, 'windows': 52, 'cannot': 53, 'table': 54, 'call': 55, 'files': 56, 'ajax': 57, 'check': 58, 'time': 59, 'name': 60, 'two': 61, 'ruby': 62, 'getting': 63, 'date': 64, 'web': 65, 'make': 66, 'user': 67, 'input': 68, 'xml': 69, 'app': 70, 'another': 71, 'element': 72, 'custom': 73, 'event': 74, 'api': 75, 'access': 76, 'url': 

In [0]:
DICT_SIZE = 5000
WORDS_TO_INDEX = wordsToIndex
INDEX_TO_WORDS = indexOfWords
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
  words = text.split(" ")
  result_vector = np.zeros(dict_size)
  for w in words:
    for i,word in enumerate(words_to_index.keys()):
      if(word==w):
        result_vector[i] = 1
        
  return result_vector

In [0]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

Execute the test_text_prepare function <br>
*<u>Note:</u> You should pass the above test to ensure BOW is working correctly!*

In [40]:
test_my_bag_of_words()

'Basic tests are passed.'

Now apply the implemented function to all samples (this might take up to a minute):

In [0]:
from scipy import sparse as sp_sparse

In [0]:
X_train = trainCleanDf['title']
X_test = testCleanDf['title']
X_val = validationCleanDf['title']

In [44]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (100000, 5000)
X_val shape  (30000, 5000)
X_test shape  (20000, 5000)


As you might notice, we transform the data to sparse representation, to store the useful information efficiently. There are many types: of such representations, however sklearn algorithms can work only with  csr matrix, so we will use this one.<br>
<u>Documentations on sparse matrix:</u> <br>
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) 
(https://docs.scipy.org/doc/scipy/reference/sparse.html)




For the 11th row in *X_train_mybag* find how many non-zero elements it has. In this task the answer (variable *non_zero_elements_count*) should be a number, e.g. 20.

In [45]:
X_train[11]

'ipad selecting text inside text input tap'

In [46]:
print(X_train_mybag[11].count_nonzero())

6


## Task 4 - TF-IDF (5 points)

The second approach extends the bag-of-words framework by taking into account total frequencies of words in the corpora. It helps to penalize too frequent words and provide better features space. 

Implement function *tfidf_features* using class  from *scikit-learn*. Use *train* corpus to train a vectorizer. Don't forget to take a look into the arguments that you can pass to it. We suggest that you filter out too rare words (occur less than in 5 titles) and too frequent words (occur more than in 90% of the titles). Also, use bigrams along with unigrams in your vocabulary. 

## Write a function which takes x_train, x_val and x_test as input and return the tf-idf features of the same and the vocabulary

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vect = TfidfVectorizer(min_df=5,max_df=0.9,ngram_range=(1,2),token_pattern= '(\S+)', max_features=5000)

In [0]:
def tfidf_features(X_train, X_val, X_test):
  X_train_tfidf = vect.fit_transform(X_train)
  X_val_tfidf = vect.fit_transform(X_val)
  X_test_tfidf = vect.fit_transform(X_test)
  return X_train_tfidf, X_val_tfidf, X_test_tfidf, vect.vocabulary_

In [0]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

Print the index of string "C#" in the vocabulary

In [51]:
print(list(tfidf_reversed_vocab.keys())[list(tfidf_reversed_vocab.values()).index('c#')])

436


## Task 5: Classification (15 points)
MultiLabel classifier

As we have noticed before, in this task each example can have multiple tags. To deal with such kind of prediction, we need to transform labels in a binary form and the prediction will be a mask of 0s and 1s. For this purpose it is convenient to use MultiLabelBinarizer from sklearn. <br>
<u>Documentation:</u> <br>
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
y_train = trainCleanDf['tags']
y_val = validationCleanDf['tags']

In [56]:
y_train.head()

0                       ['r']
1            ['php', 'mysql']
2                      ['c#']
3    ['javascript', 'jquery']
4                    ['java']
Name: tags, dtype: object

In [59]:
sortedTags = sorted(tagDict.keys())
print(sortedTags)

['.net', 'ajax', 'algorithm', 'android', 'angularjs', 'apache', 'arrays', 'asp.net', 'asp.net-mvc', 'c', 'c#', 'c++', 'class', 'cocoa-touch', 'codeigniter', 'css', 'csv', 'database', 'date', 'datetime', 'django', 'dom', 'eclipse', 'entity-framework', 'excel', 'facebook', 'file', 'forms', 'function', 'generics', 'google-maps', 'hibernate', 'html', 'html5', 'image', 'ios', 'iphone', 'java', 'javascript', 'jquery', 'json', 'jsp', 'laravel', 'linq', 'linux', 'list', 'loops', 'maven', 'mongodb', 'multithreading', 'mysql', 'node.js', 'numpy', 'objective-c', 'oop', 'opencv', 'osx', 'pandas', 'parsing', 'performance', 'php', 'pointers', 'python', 'python-2.7', 'python-3.x', 'qt', 'r', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'ruby-on-rails-3', 'selenium', 'servlets', 'session', 'sockets', 'sorting', 'spring', 'spring-mvc', 'sql', 'sql-server', 'string', 'swift', 'swing', 'twitter-bootstrap', 'uitableview', 'unit-testing', 'validation', 'vb.net', 'visual-studio', 'visual-studio-2010', 'wcf', '

In [61]:
mlb = MultiLabelBinarizer(classes=sortedTags)
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:951: UserWarning: unknown class(es) [' ', '#', "'", '+', ',', '-', '.', '0', '1', '2', '3', '5', '7', '[', ']', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y'] will be ignored
  .format(sorted(unknown, key=str)))


In [64]:
y_train[0].nonzero()

(array([66]),)

In [67]:
list(sortedTags)[66]

'r'

In this task we suggest to use One-vs-Rest approach, which is implemented in [OneVsRestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) class. In this approach *k* classifiers (= number of tags) are trained. As a basic classifier, use [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). It is one of the simplest methods, but often it performs good enough in text classification tasks. It might take some time, because a number of classifiers to train is large.

**OneVsRest multi-label strategy**

The Multi-label algorithm accepts a binary mask over multiple labels. The result for each prediction will be an array of 0s and 1s marking which class labels apply to each row input sample.

**Logistic Regression & SVM**

OneVsRest strategy can be used for multi-label learning, where a classifier is used to predict multiple labels for instance. LR & SVM supports multi-class, but we are in a multi-label scenario, therefore, we wrap classifiers in the OneVsRestClassifier.

*If you want to learn more about OneVsRest, check out these links:*
- *https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5*
- *https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff*
- *https://medium.com/coinmonks/multi-label-classification-blog-tags-prediction-using-nlp-b0b5ee6686fc*

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

In [0]:
def train_classifier(X_train, y_train):
    # Create and fit LogisticRegression and LinearSVC wraped into OneVsRestClassifier.

    ######################################
    ######### YOUR CODE HERE #############
    ###################################### 
    
    return clf  ### clf is the model

Train the classifiers for different data transformations: *bag-of-words* and *tf-idf*.
classifier_mybag = model for 

In [0]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

Now you can create predictions for the data. You will need two types of predictions: labels and scores.

In [0]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

Now, we would need to compare the results of different predictions, e.g. to see whether TF-IDF transformation helps or to try different regularization techniques in logistic regression. For all these experiments, we need to setup evaluation procedure. 

## Evaluation (10 points)

To evaluate the results we will use several classification metrics:
 - [Accuracy](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
 - [F1-score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
 - [Area under ROC-curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
 - [Area under precision-recall curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score) 
 
Make sure you are familiar with all of them. If you want a refresher, you can click the link to their documentation

## Import the necessary libraries for the above metrics

Define the function *print_evaluation_scores* which takes y_val and predicted as input calculates and prints the following output:
 - *accuracy*
 - *F1-score - Average = 'weighted'* 
 - *Precision - Average = 'macro'*

In [0]:
def print_evaluation_scores(y_val, predicted):
    


In [0]:
print('Bag-of-words')
print_evaluation_scores(y_val, y_val_predicted_labels_mybag)
print('Tfidf')
print_evaluation_scores(y_val, y_val_predicted_labels_tfidf)

You might also want to plot some form of the [ROC curve](http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc) for the case of multi-label classification. The input parameters for the roc curve are:
 - true labels
 - decision functions scores
 - number of classes

Import the roc_auc function from the metrics.py file provided

## Task 4 (MultilabelClassification) - Optional 
** Once we have the evaluation set up, we suggest that you experiment a bit with training your classifiers. We will use *F1-score weighted* as an evaluation metric. Our recommendation:
- compare the quality of the bag-of-words and TF-IDF approaches and chose one of them.
- for the chosen one, try *L1* and *L2*-regularization techniques in Logistic Regression with different coefficients (e.g. C equal to 0.1, 1, 10, 100).



In [0]:
######################################
######### YOUR CODE HERE #############
######################################

When you are happy with the quality, create predictions for *test* set.